In [1]:
import numpy as np
import math
import json
import random

def waveforms(N_A, N_g, N_f, t0_tf, T, B, trials, seedn=1, inputfile="input", 
              phi0=0, A0=1, Af=50, g0=0, gf=2, F0=90, Ff=110, N=10000):

    # initalizes the arrays which span parameter space, and their lengths
    A_RANGE=np.linspace(A0,Af,N_A)
    G_RANGE=np.linspace(g0,gf,N_g)
    F_RANGE=np.linspace(F0,Ff,N_f)

    # number of parameters available
    A_LEN, G_LEN, F_LEN = N_A, N_g, N_f
    
    waveform_data={}
    for j in range(trials):
        waveform_data.update({j:[[],[]]})
        
        # calculates random indice for each parameter (A, f, g)
        A_RAN=random.randint(0,A_LEN-1)
        G_RAN=random.randint(0,G_LEN-1)
        F_RAN=random.randint(0,F_LEN-1)
        
        # calculates random parameters A, f, g
        A, gamma, f = A_RANGE[A_RAN], G_RANGE[G_RAN], F_RANGE[F_RAN]
        
        dt=T/N # time resolution

        t0=(T-t0_tf)*np.random.random(1)[0]  # randomly generate start time
        START_INDEX=math.floor(t0/dt)        # find index associated with time

        ##NOTE: using 't0' instead of some multiple of dt may cause issues later
        
        SIG_LEN = (math.floor(t0_tf/dt)+1 if (t0_tf != T) else N) # calculate # of indexes signal takes
        INJECTED = np.zeros(N)                 # initalize injected signal, with N size array of zeroes
        for i in range(SIG_LEN):
            INJECTED[START_INDEX + i]=t0+i*dt       # fill in injected signal with its time stamps

        w = 2 * np.pi * f
        
        # replace timestamps with their displacement values
        SR = INJECTED[START_INDEX : START_INDEX+SIG_LEN][:]
        INJECTED[START_INDEX : START_INDEX+SIG_LEN] = A*np.sin(w*(SR-t0) + phi0)*np.exp(-gamma*(SR-t0))
        
        # Purposed for Quadrature Sum
        D_i = [] # list of each differently seeded waveform
        for n in range(seedn):
            np.random.seed(seed = n)
            NOISE = np.random.normal(scale=(B/(math.sqrt(3)*2)), size=N)  # Noise!
            D_i.append(list(NOISE + INJECTED))  # complete data!
        
        # gets parameters and data for each trial, stuffs it into dictionary
        parameters = [A, f, gamma, t0]
        waveform_data[j][0], waveform_data[j][1] = parameters, D_i
        
    # each trial has list of parameters used and list of data values
    with open("{}-waveform_data.json".format(inputfile) , "w") as f:
        json.dump(waveform_data, f, indent=2, sort_keys=True)

In [2]:
#  N_A, N_g, N_f, t0_tf, T, B, trials
waveforms(4, 4, 4, 4, 10, 0, 10, N=250, seedn=1)

In [71]:
import json
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import math
import time

# Produces a template given a position in parameter space
def template(A, f, gamma, duration, dt):
    t = np.arange(0, duration + dt, dt)
    w = 2 * np.pi * f
    return A * np.sin(w*t)*np.exp(-gamma*t)

# Produces a cross corelation function given a data input and a template in parameter space
def CrossCorrelation(data, template, dt):
    ii = 0
    M = []

    while len(data[ii:]) >= len(template):
        M.append(np.sum((data[ii: len(template) + ii] * template)))
        ii+=1

    return M

# Produces chi square at each "slide"
def ChiSquare(data, template, dt):
    ii = 0
    C = []

    while len(data[ii:]) >= len(template):
        C.append(-1 * np.sum((data[ii: len(template) + ii] - template) ** 2))
        ii += 1

    return C

def modulator(rho_ij, D, dt):

    rho_mod_D, RHO_ij = [] , rho_ij[:]

    dn , L = math.floor(2*D/dt) , len(RHO_ij)

    for i in range(0,L-(L%dn),dn):
        rho_mod_D.append(max(RHO_ij[i:i+dn]))

    if (L-(L%dn)) != L :
        rho_mod_D.append(max(RHO_ij[L-(L%dn):L]))

    return rho_mod_D

def statudio(trialn, D, N_A, N_g, N_f, t0_tf, T, trials, run1 = True,
                  seedn=1, inputfile="input", A0=1, Af=50, g0=0, gf=2,
                  F0=90, Ff=110, N_t=10000):

    # initalizes the arrays which span parameter space, and their lengths
    A_RANGE=np.linspace(A0,Af,N_A)
    G_RANGE=np.linspace(g0,gf,N_g)
    F_RANGE=np.linspace(F0,Ff,N_f)

    A_LEN, G_LEN, F_LEN = N_A, N_g, N_f

    # constructs timestep resolution, and saves N and t0/tf internally 
    N, dt = N_t, T/N_t

    # constructs time range to pick injected signal start time from/ corresponding length 
    t_RANGE=np.linspace(0,T-(t0_tf),int(N_t*(1-((t0_tf)/(T)))))
    t_LEN=len(t_RANGE)

    # initialize arrays for various data/cross-correlations/chi-squares 
    noise = []

    # constructs all templates which correspond to points in the parameter space
    TEMPLATES_AFG=[ template( A, f, g, t0_tf, dt) for A in A_RANGE 
                   for g in G_RANGE for f in F_RANGE]

    AFG_PAIR=[ [A, f, g] for A in A_RANGE 
                   for g in G_RANGE for f in F_RANGE]

    # Reads waveform data file 
    with open("{}-waveform_data.json".format(inputfile),"r") as f: 
        waveform_data = json.load(f)

    waveform_data = waveform_data[str(trialn)] # trialn's parameters and differently seeded data
    
    output={}

    output.update({trialn:[[],[],[],[]]})

    # isolates random a-g-f pair / data set 
    temp_AGFT, data = waveform_data[0], waveform_data[1]

    noise.append(data) 

    output[trialn][0], output[trialn][1] = temp_AGFT, data  # stores random a-g-f pair / data set 

    Quad_CRS = []
    Quad_CHI = []
    
    # performs base static calculation across parameter space
    # Quadrature Sum
    for n in range(seedn): # Use seedn as index for data

        CRS_COR, CHI_SQR = [[],[]]

        for temp in TEMPLATES_AFG:

            CC_dh = list(CrossCorrelation(data[n], temp, dt))
            CRS_COR.append(CC_dh)

            CS_dh = list(ChiSquare(data[n], temp, dt))
            CHI_SQR.append(CS_dh)

        Quad_CRS.append(CRS_COR) # now a 3d list of seedn statistics, with 2d list statistics per waveform
        Quad_CHI.append(CHI_SQR)

    CRS_COR = np.sum(np.array(Quad_CRS) ** 2, axis = 0) ** .5 # Quadrature sum of each seed's statistic
    CHI_SQR = np.sum(np.array(Quad_CHI) ** 2, axis = 0) ** .5

    # stores base statistics to attribute
    cross_cor = CRS_COR
    chi = CHI_SQR
    output[trialn][2], output[trialn][3] = CRS_COR.tolist(), CHI_SQR.tolist() # store quadrature summed based statistics
    
#calculates test statistic, stores it internally,
#and returns a copy of it as a dictionary 
# trying connect stat and anlysis function
    
    # counts number of tempates in parameter space
    PSPACE_LEN = len(AFG_PAIR)
    
    # String to equation!
    stats = ["CC_IJ","CS_IJ","CC_IJ/abs((1+CS_IJ))"]

    # initalizes rho statistic dictionary/stat outputs collector
    RHO = {}
    stat_outputs = []
    
    for stat in stats: # objective is that the appropriate jsons holds multiple stat results

        # indexed to loops through parameter space templates and
        # calculates each rho_ij given template j
        rho_i = []
        for j in range(PSPACE_LEN):
            CC_IJ = np.array(cross_cor[j][:])
            CS_IJ = np.array(chi[j][:])

            # Evaluates string (Exec gave issues... eval is the same concept though)
            p = eval(stat)
            rho_i.append(list(p))
        stat_outputs.append(rho_i) # stat_outputs is 3d list holding a 2d list of a stat's outputs for each template
    RHO.update({ trialn : stat_outputs })

    if (2*D >= dt):
        
        # gets the length of linearized template space
        TEMP_LEN=len(cross_cor) # number of templates 

        RHO_MOD={}
        MAX_OS={}
        MAX_BG_TEMP={}
        BG_VALS_IJ, FG_VAL_IJ = [], []
        pot_thresholds = {}
        
        # seperates fg value from bg value
        T0_2D = math.floor(output[trialn][0][3]/(2*D))

        for j in range(TEMP_LEN):
            MAX_BG_TEMP.update({ j : list(np.zeros(len(stats)))})

        for stat in range(len(stats)):
            
            pot_thresholds.update({stat:[]})
            
            BG_VALS_ij, FG_VAL_ij = [], []
            for j in range(TEMP_LEN):

                # calculates bg values + fg values
                BG_VALS_ij.append(modulator(RHO[trialn][stat][j][:],D,dt))

                FG_VAL_ij.append(BG_VALS_ij[j].pop(T0_2D))
                    
            BG_VALS_IJ.append(BG_VALS_ij) # 3d list
            FG_VAL_IJ.append(FG_VAL_ij) # 2d list
            
            pot_thresholds[stat] += FG_VAL_ij # for each stat key, forground values of trialn are added
        RHO_MOD.update({ trialn: [ BG_VALS_IJ, FG_VAL_IJ ] }) # these are the peaks we look for
        
        stat_dict = {}
        for stat in range(len(stats)):
            stat_dict.update({stat:[tuple(output[trialn][0][0:3]),max(RHO_MOD[trialn][1][stat])] })
        MAX_OS.update({trialn: stat_dict}) # stores t0 and forground peaks for each trial

        for stat in range(len(stats)):
            for j in range(TEMP_LEN):
                new_max=max(RHO_MOD[trialn][0][stat][j]) # max ofsource peak for each template

                MAX_BG_TEMP[j][stat] = new_max # every value in MAX_BG_TEMP dictionary changes from 0 to that templates max
    else: 
        print("invalid D; it is required that 2*D >= T/N")

    # All jsons below will serve post processing

    # output holds trialn's parameters, data, cross-correlation, chi-square
    with open("output_folder/output_{}.json".format(trialn), "w") as f: 
        json.dump(output, f, indent=2, sort_keys=True)

    # RHO_MOD holds trialn's background and forground values
    with open("Peaks_folder/Peaks_{}.json".format(trialn), "w") as f:
        json.dump(RHO_MOD, f, indent=2, sort_keys=True)

    # MAX_OS holds trialn's max onsource peak
    with open("Max_OS_folder/Max_OS_{}.json".format(trialn), "w") as f:
        json.dump(MAX_OS, f, indent=2, sort_keys=True)
    
    # MAX_BG_TEMP and pot_thresholds were devised wrong. They used to rewrite the same file with
    # changed or more value in every run. This won't work when parallelized as some runs may finish
    # faster than even the first. They are now made for each trial, but because of their different form
    # compared to the simple first three, weirder json combiners must be used on them

    with open("Max_BG_TEMP_folder/Max_BG_TEMP{}.json".format(trialn), "w") as f: # Merged
        json.dump(MAX_BG_TEMP, f, indent=2, sort_keys=True)
        
    with open("thresholds_folder/thresholds{}.json".format(trialn), "w") as f:
        json.dump(pot_thresholds, f, indent=2, sort_keys=True)
    
    # Now only the essentials file is made when run1 is True
    # essentials holds all the values that don't change for each trialn run
    if run1 == True:
        essent = {"essentials":[A_LEN,F_LEN,G_LEN,list(F_RANGE),list(A_RANGE),list(G_RANGE),AFG_PAIR,trials]}
        with open("essentials.json", "w") as f:
            json.dump(essent, f, indent=2, sort_keys=True)
    
    return(MAX_BG_TEMP)

In [72]:
# (trialn, D, N_A, N_g, N_f, t0_tf, T, trials, run1 = True,
# seedn=1, inputfile="input", A0=1, Af=50, g0=0, gf=2, F0=90, Ff=110, N_t=10000):

statudio(2, .02, 4, 4, 4, 4, 10, 10, N_t=250, seedn=1, run1=False)

{0: [0.2444605573891365, 55.14750191152992, 0.004430464408788846],
 1: [9.270460463768025, 73.54936943627463, 0.24592413694069895],
 2: [9.270460463768124, 73.77826555342466, 0.2414136025663782],
 3: [0.2444605573892886, 55.15502792236894, 0.0043557647916079145],
 4: [0.2283342697424941, 14.41607797688298, 0.01572133249683823],
 5: [5.521577987297916, 25.033496375496302, 0.8279103862314042],
 6: [5.521577987297962, 23.33621494790189, 1.4106468031564248],
 7: [0.22833426974250715, 14.43718674555614, 0.014973051718731784],
 8: [0.21677525642441603, 9.721094212214856, 0.02192378107971594],
 9: [3.726209597885268, 16.787933055037154, 1.3691086177901375],
 10: [3.7262095978852816, 15.573399184035587, 2.787370997212679],
 11: [0.21677525642442608, 9.754777591898128, 0.020156182177837774],
 12: [0.20564736949852033, 8.109160945612786, 0.02468449544063521],
 13: [2.981533894359787, 13.12319899734397, 1.397137224522042],
 14: [2.9815338943597727, 12.804930216359569, 2.3774302232618436],
 15: [0

In [78]:
import glob
import json

# works like a charm, put in your path and
# make sure to include the path of the produced json in their merge name

# This function will be run on the jsons of output, RHO_MOD, and MAX_OS
# It updates all trial dictionaries, and since each dictionary has a different key 
# (which indicates the trial used), then will be added, till 1 dictionary for all
# trials is made

def json_stack_keys(jsons_path, merge_path_name):
    
    # Include last / at end of path!
    files = glob.glob("{}*.json".format(jsons_path))

    count = 0
    for file in files:
        if count == 0:
            count += 1
            with open(file, "r") as f:
                C_dictionary = json.load(f)
        else:
            with open(file, "r") as f:
                C_dictionary.update(json.load(f))

    with open('{}.json'.format(merge_path_name), 'w') as f:
        json.dump(C_dictionary, f, indent=2)

# This function will be run on the jsons of MAX_BG_TEMP
# MAX_BG_TEMP always has the same keys, as they are the templates used per trial
# As such we update the max offsource value for each stat of each template of each trial
# We therefore have the crosses across all trials

def json_update_components(jsons_path, merge_path_name):
    
    with open("Merged_Peaks.json", "r") as f:
        RHO_MOD = json.load(f)
    
    # way of getting the first key of a dictionary, used to get tempn and statn quick and dirty
    get_first_key = []
    get_first_key += RHO_MOD.keys()
    dict_key = get_first_key[0]
    
    # way of getting tempn and statn from RHO_MOD
    tempn = len(RHO_MOD[dict_key][0][0])
    statn = len(RHO_MOD[dict_key][0])
    
    # Include last / at end of path!
    files = glob.glob("{}*.json".format(jsons_path))

    count = 0
    for file in files:
#         trialn = "" # method of obtaining this file's trialnumber through its name
#         for character in file:
#             if character.isdigit():
#                 trialn += str(character)
#         trialn = int(trialn)

        if count == 0:
            count += 1
            with open(file, "r") as f:
                C_dictionary = json.load(f)
        else:
            with open(file, "r") as f:
                C_dictionary_new = json.load(f)
                
            for stat in range(statn):
                for j in range(tempn):
                
                    if (C_dictionary_new[str(j)][stat] > C_dictionary[str(j)][stat]): # every value in MAX_BG_TEMP dictionary changes from 0 to that templates max
                        C_dictionary[str(j)][stat] = C_dictionary_new[str(j)][stat]

    with open('{}.json'.format(merge_path_name), 'w') as f:
        json.dump(C_dictionary, f, indent=2)

# This function will be run on the jsons of pot_threshold
# pot_thresholds is a 2d list, where each list is a stat's onsource peaks for a trial
# The lists will have each trials values added to them until we havethe onsource peaks for all trials 

def json_list_append(jsons_path, merge_path_name):

    # Include last / at end of path!
    files = glob.glob("{}*.json".format(jsons_path))

    count = 0
    for file in files:
        if count == 0:
            count += 1
            with open(file, "r") as f:
                C_dictionary = json.load(f)
        else:
            with open(file, "r") as f:
                C_dictionary_new = json.load(f)
        
            for i in C_dictionary:
                C_dictionary[str(i)] += C_dictionary_new[str(i)]

    with open('{}.json'.format(merge_path_name), 'w') as f:
        json.dump(C_dictionary, f, indent=2)

In [79]:
json_stack_keys("output_folder/", "Merged_output")

In [80]:
json_stack_keys("Peaks_folder/", "Merged_Peaks")

In [81]:
json_stack_keys("Max_OS_folder/", "Merged_output")

In [82]:
json_update_components("MAX_BG_TEMP_folder/", "Merged_MAX_BG_TEMP")

In [83]:
json_list_append("thresholds_folder/", "Merged_thresholds")